# **Preprocesamiento de datos**

In [1]:
# Librerias
import pandas as pd
import numpy as np

Para el caso de estudio, se preprocesan los datos dependiendo del contexto de cada variable. Intentando obtener el conjunto de variables que aporten en mayor medida a la clasificación de casos. 

## **Reducción del dataset a la variable objetivo**
Definición de la variable objetivo `default`.\
    Se fija: (1) _Default_ (0) _Fully Paid_

In [2]:
# Lectura del dataset original
file = r"C:\Users\Hp\Downloads\accepted_2007_to_2018Q4.csv\accepted_2007_to_2018Q4.csv"
df = pd.read_csv(file, low_memory=False)
print("Dimensiones del DataFrame original:", df.shape)

Dimensiones del DataFrame original: (2260701, 151)


In [3]:
# Definición de la variable objetivo `default`
df = df[df['loan_status'].isin(['Charged Off', 'Fully Paid'])].copy()
df['default'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)
df.drop(columns=['loan_status'], inplace=True)

In [4]:
print("Dimensiones del DataFrame con la variable objetivo:", df.shape)

Dimensiones del DataFrame con la variable objetivo: (1345310, 151)


In [5]:
# Ordenar columnas en orden alfabético
df = df.reindex(sorted(df.columns), axis=1)

## **Tratamiento de variables**

### **Reemplazo de valores inválidos con NA**
Cambiar códigos especiales (-1, 999) por `NaN`.

In [6]:
(df == -1).sum().sum(), (df == 999).sum().sum()
cols_con_valores = df.columns[(df.isin([-1, 999])).any()]
df[cols_con_valores] = df[cols_con_valores].replace([-1, 999], np.nan)

### **Transformación y agregación**
Ajustar formatos y creación de variables derivadas

* Comportamiento o decisiones de los inversores.

In [7]:
# Variables de decisiones de los inversores
# Lista de columnas que terminan en _inv
inv_cols = [col for col in df.columns if col.endswith('_inv')]
df.drop(columns=inv_cols, inplace=True)

* Variables de rangos, creación de una variable derivada con la media

In [8]:
# Buscar todas las columnas que terminan en _low
low_cols = [col for col in df.columns if col.endswith('_low')]
for low_col in low_cols:
    high_col = low_col.replace('_low', '_high')
    if high_col in df.columns:
        mean_col = low_col.replace('_low', '_mean')
        df[mean_col] = np.round((df[low_col] + df[high_col]) / 2, 2)
        df.drop(columns=[low_col, high_col], inplace=True)

### **Codificación de variables categóricas**

* **Extracción de codigo**\
Se extrae el código de la variable

In [9]:
# Extraer el número de sub_grade y obtener el número de sub_grade
df['sub_grade_num'] = df['sub_grade'].str[1].astype(int)
df.drop(columns=['sub_grade'], inplace=True)

* **Codificación Dummy**

Conversion a variables dummy:
|   Variable    |   Categorías  |   Codificación |
|---------------|---------------|----------------|
| `debt_settlement_flag`  | 'Y' - 'N'     | 1 - 0          |
| `term`        | '36 months' - '60 months'  | 0-1 |
| `initial_list_status` | 'w' - 'f'          | 0-1 |
| `application_type`    | 'Individual' - 'Joint App' | 0-1          |
| `disbursement_method` | 'Cash' - 'DirectPay'` | 0-1                |
| `verification_status` | 'Not Verified' - 'Source Verified' - 'Verified' | 0-1-2                   |
| `grade` | 'A' - 'B' - 'C' - 'D' - 'E' - 'F' - 'G' | 0-1-2-3-4-5-6                 |

In [10]:
#Variables binarias con Y/N o y/n
df['debt_settlement_flag'] = df['debt_settlement_flag'].apply(lambda x: 1 if x=='Y' else 0)

# Diccionarios de mapeo para variables categoricas
maps = {
    'term': {' 36 months': 0, ' 60 months': 1},
    'initial_list_status': {'w': 0, 'f': 1},
    'application_type': {'Individual': 0, 'Joint App': 1},
    'disbursement_method': {'Cash': 0, 'DirectPay': 1},
    'verification_status': {'Not Verified': 0, 'Source Verified': 1,'Verified': 2},
    'grade': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6},
    'emp_length':{'10+ years': 10,'9 years': 9,'8 years': 8,'7 years': 7,'6 years': 6,
        '5 years': 5,'4 years': 4,'3 years': 3,'2 years': 2,'1 year': 1,'< 1 year': 0}
}

# Aplicar el mapeo
for col, mapping in maps.items():
    df[col + '_num'] = df[col].map(mapping)
    
df.drop(columns=maps.keys(), inplace=True)


### **Destring a fecha mm/yyyy**
Transformación de variables tipo MM-YYYY en String, a dos variables tipo DateTime '_year' y '_month'

In [11]:
# Convertir columnas de fecha y extraer año y mes
date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

for col in date_cols:
    df[col] = pd.to_datetime(df[col], format="%b-%Y", errors='coerce')
# generar año y mes en un solo paso y concatenar
new_features = pd.concat(
    [df[col].dt.year.rename(col + "_year") for col in date_cols] +
    [df[col].dt.month.rename(col + "_month") for col in date_cols],
    axis=1
)

# concatenar con el dataframe original y eliminar las columnas originales
df = pd.concat([df.drop(columns=date_cols), new_features], axis=1)

### **Eliminación de variables identificadoras**
Códigos únicos que no aportan información predictiva

In [12]:
df = df.drop(columns=['id','url','emp_title','title', 'zip_code','addr_state',
                      'member_id', 'policy_code', 'desc', ])

### **Descarte de variables**

* **Variables con único valor**

In [13]:
# Filtra columnas con un único valor
cols_unicas = df.columns[df.nunique() == 1].tolist()
df = df.drop(cols_unicas, axis=1)

In [14]:
df['loan_gap'] = df['loan_amnt']-df['funded_amnt']
df.drop(columns='loan_amnt', inplace=True)

* **Variables dificles de imputar - redundantes**

In [15]:
# Eliminación de variables 

# Variales temporales dificiles de imputar
antig = ['mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op', 
         'mo_sin_rcnt_tl', 'acc_open_past_24mths']

# Limites y saldos financieros
limites = ['avg_cur_bal', 'tot_cur_bal', 'total_il_high_credit_limit', 
           'total_rev_hi_lim', 'bc_open_to_buy', 'total_bal_ex_mort', 
           'total_bc_limit']

# Utilizacion y comportamiento crediticio
utilizacion = ['bc_util', 'percent_bc_gt_75','open_acc']

# Morosidad y eventos negativos
mora = ['num_tl_120dpd_2m', 'num_tl_90g_dpd_24m', 'num_tl_30dpd', 
        'num_accts_ever_120_pd', 'tax_liens', 'pub_rec']

# Número de cuentas y actividad
activi =    ['num_rev_accts', 'num_il_tl', 'num_rev_tl_bal_gt_0','num_sats',
            'num_actv_bc_tl', 'num_bc_tl', 'num_op_rev_tl', 'num_actv_rev_tl','inq_last_6mths']
# Eliminar columnas originales de scores
df.drop(columns=antig + limites + utilizacion + mora + activi, inplace=True)


* **Eliminación de variables post-default**

In [16]:
df.drop(columns =['last_pymnt_amnt','recoveries', 'collection_recovery_fee',
                  'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
                  'last_pymnt_d_year', 'last_pymnt_d_month',
                  'last_credit_pull_d_year', 'last_credit_pull_d_month', 'last_fico_range_mean',
                  ], inplace=True)

In [17]:
print("Dimensiones luego del tratamiento inicial:", df.shape)

Dimensiones luego del tratamiento inicial: (1345310, 92)


## **Análisis de valores nulos**

In [18]:
# Conteo de variables con valores faltantes
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]
print("Variables con valores faltantes:", len(missing))

Variables con valores faltantes: 68


* **Variables con más del 50% de datos perdidos**

Se eliminaran por ser poco informativas y difíciles de imputar.

In [19]:
# Missing 50%
missing = df.isnull().sum().sort_values(ascending=False)
umbral = 0.5* len(df)
faltan_50= missing[missing > umbral]
print("Columnas con más del 50% de valores faltantes:", len(faltan_50))

df.drop(columns=faltan_50.index, inplace=True)

Columnas con más del 50% de valores faltantes: 52


* **Variables restantes con valores nulos**

In [ ]:
# Conteo de variables con valores faltantes
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]
print("Variables con valores faltantes:", len(missing))
table = pd.DataFrame(missing, columns=['missing_count'])
table['percentage'] = (table['missing_count'] / len(df)) * 100
table.sort_values(by='percentage', ascending=False, inplace=True)
table

Variables con valores faltantes: 16
                            missing_count  percentage
mths_since_recent_inq              174071   12.939100
emp_length_num                      78511    5.835904
pct_tl_nvr_dlq                      67681    5.030885
tot_coll_amt                        67561    5.021965
num_tl_op_past_12m                  67527    5.019438
tot_hi_cred_lim                     67527    5.019438
mths_since_recent_bc                60221    4.476366
num_bc_sats                         55841    4.150791
mort_acc                            47281    3.514506
revol_util                            857    0.063703
pub_rec_bankruptcies                  697    0.051810
dti                                   414    0.030774
chargeoff_within_12_mths               56    0.004163
collections_12_mths_ex_med             56    0.004163
revol_bal                              40    0.002973
delinq_amnt                             1    0.000074


In [21]:
print("Dimensiones finales del data frame:", df.shape)

Dimensiones finales del data frame: (1345310, 40)


----------------

In [22]:
# Renombrar variables
variable_dict = {
    "prestamo": {
        "funded_amnt": "monto_aprobado",
        "int_rate": "tasa_interes",
        "installment": "cuota_mensual",
        "term_num": "plazo_meses",
        "purpose": "motivo_prestamo",
        "disbursement_method_num": "metodo_desembolso",
        "initial_list_status_num": "estado_inicial_listado",
        "application_type_num": "tipo_aplicacion",
        "loan_gap" : 'diferencia_monto_solicitado',
    },
    "prestatario": {
        "annual_inc": "ingreso_anual",
        "home_ownership": "tipo_vivienda",
        "emp_length_num": "antiguedad_laboral",
        "verification_status_num": "estado_verif_ingreso"
    },
    "historial_crediticio": {
        "fico_range_mean": "promedio_fico",
        "earliest_cr_line_year": "anio_apertura_credito",
        "earliest_cr_line_month": "mes_apertura_credito",
        "mort_acc": "cuentas_hipotecarias",
        "total_acc": "total_cuentas_credito",
        "num_bc_sats": "cuentas_tarjeta_credito",
        "revol_bal": "saldo_revolvente",
        "revol_util": "uso_credito_revolvente",
        "tot_hi_cred_lim": "limite_credito_total"
    },
    "comportamiento_prestatario": {
        "mths_since_recent_inq": "meses_ultima_consulta",
        "mths_since_recent_bc": "meses_tarjeta_nueva",
        "mths_since_recent_revol_delinq": "meses_mora_revolvente",
        "num_tl_op_past_12m": "lineas_credito_12m",
        "pct_tl_nvr_dlq": "porcentaje_sin_moras"
    },
    "morosidad_riesgo": {
        "delinq_2yrs": "moras_2_year",
        "mths_since_last_delinq": "meses_ultima_mora",
        "delinq_amnt": "monto_mora",
        "chargeoff_within_12_mths": "castigos_12m",
        "collections_12_mths_ex_med": "cobranzas_12m",
        "acc_now_delinq": "cuentas_en_mora_actual",
        "tot_coll_amt": "monto_total_cobranzas",
        "pub_rec_bankruptcies": "quiebras_publicas",
        "debt_settlement_flag": "tuvo_acuerdo_pago"
    },
    "tecnicas_control": {
        "grade_num": "categoria_credito",
        "sub_grade_num": "subcategoria_credito",
        "issue_d_year": "anio_emision_prestamo",
        "issue_d_month": "mes_emision_prestamo"
    }
}
# Combinar todos los subdiccionarios en uno solo y renombrar columnas
rename_map = {k: v for group in variable_dict.values() for k, v in group.items()}
df.rename(columns=rename_map, inplace=True)

## Información final del dataset

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1345310 entries, 0 to 2260697
Data columns (total 40 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   cuentas_en_mora_actual       1345310 non-null  float64
 1   ingreso_anual                1345310 non-null  float64
 2   castigos_12m                 1345254 non-null  float64
 3   cobranzas_12m                1345254 non-null  float64
 4   tuvo_acuerdo_pago            1345310 non-null  int64  
 5   default                      1345310 non-null  int64  
 6   moras_2_year                 1345310 non-null  float64
 7   monto_mora                   1345309 non-null  float64
 8   dti                          1344896 non-null  float64
 9   monto_aprobado               1345310 non-null  float64
 10  tipo_vivienda                1345310 non-null  object 
 11  cuota_mensual                1345310 non-null  float64
 12  tasa_interes                 1345310 non-null  

Al finalizar el preprocesamiento se obtienen 39 variables de estudio, más adelante se realiza una selección de variables de mayor importancia para la construcción del modelo. 

In [ ]:
# Transformar a csv
df.to_csv("data_lend.csv", index=False)